hi!

# Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from dateutil.parser import isoparse, parse
from datetime import datetime, timedelta, timezone

In [2]:
tz = timezone.utc

In [3]:
TI_SHEET_LOCATION = "/home/elia/.ti-sheet"

# Load the data!!!
with open(TI_SHEET_LOCATION) as file:
    data = json.load(file)["work"]

In [4]:
# Parse the strings in the data and store in memory
def parseDates(data):
    for x in data:
        x["start"] = isoparse(x["start"])
        if "end" not in x:
            x["end"] = datetime.now(tz=timezone.utc)
        else:
            x["end"] = isoparse(x["end"])
    
    return data

data = parseDates(data)

In [20]:
# Function to get the total amount of minutes!
def getTotalMinutes(data, startoflog, endoflog=0):
    # endoflog is an optional parameter, if unspecified will default to "right now" (see few lines down for where it is assigned)

    # get current datetime, and store if no endoflog given
    now = datetime.now(tz=timezone.utc)
    if (endoflog == 0): endoflog = now # default assignment for optional here, because we need it to be equal to now

    # for storing amount of time spent on each activity
    timedata = {}

    # loop through every item in ti_sheet,
    for x in data:
        xstart = x["start"]
        xend = x["end"]
        
        # if before or after this period of time, skip
        if (xstart < startoflog): continue 
        elif (xend > endoflog): continue

        # calculate difference as total minutes
        difference = (xend - xstart)
        difference = int(difference / timedelta(minutes=1))

        # save total time(s) into dictionary
        if x["name"] not in timedata: timedata[x["name"]] = 0
        timedata[x["name"]] += difference

    # return
    return timedata

# Display data
First by minutes, then in plots

In [21]:
# Get start of day and start of week!
now = datetime.now(tz=tz)
startofday = datetime(now.year, now.month, now.day, tzinfo=now.tzinfo)
startofweek = datetime(now.year, now.month, now.day - now.weekday(), tzinfo=now.tzinfo)

# Define custom dates here if you wish
start = "01/02/2025"
end = "01/02/2026"
start = parse(start, dayfirst=True).replace(tzinfo=tz)
end = parse(end, dayfirst=True).replace(tzinfo=tz)

Below, execute the cell you want to use

In [7]:
# Use custom dates, defined above
activity = getTotalMinutes(data, start, end)

In [12]:
# Use current day
activity = getTotalMinutes(data, startofday)

In [14]:
# Use current week
activity = getTotalMinutes(data, startofweek)

Then, how do you want to display it?

In [22]:
# Print as JSON, as amount of minutes
print(activity)

{'comp1004': 95, 'comp1043': 143, 'coding': 9, 'comp1009': 65}


In [23]:
# Pretty print amounts of hours and minutes
print()
totalmins = 0
for x in activity:
    hours = int(activity[x] / 60)
    minutes = activity[x] % 60
    totalmins += activity[x]
    print(f"{x} : {hours} hour{'' if hours == 1 else 's'} and {minutes} minutes")
hours = int(totalmins/60)
minutes = totalmins % 60
print(f"\nTotal : {hours} hour{'' if hours == 1 else 's'} and {minutes} minutes")


comp1004 : 1 hour and 35 minutes
comp1043 : 2 hours and 23 minutes
coding : 0 hours and 9 minutes
comp1009 : 1 hour and 5 minutes

Total : 5 hours and 12 minutes
